<h5 style="color:rgba(0, 0, 178, 0.6);">This code hidden warrnings</h5>

In [3]:
import warnings
warnings.filterwarnings('ignore')

<h5 style="color:rgba(0, 0, 178, 0.6);">Install librarys that we need</h5>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("bmh")
from tensorflow import keras
from sklearn.cluster import KMeans
import random
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
tf.__version__

'2.15.0'

<h5 style="color:rgba(0, 0, 178, 0.6);">Read the data from data set</h5>

In [4]:
df = pd.read_csv("../../DATA/Data_increase.csv").drop(["Unnamed: 0","Unnamed: 0.1"], axis=1)
df["LLD-LLS"] = df.log_LLD - df.log_LLS
df["GR-GKUT"] = df.GR - df.GKUT
df

,DEPT,GR,GKUT,NPHI,PE,RHOB,DRHO,well,log_LLS,log_LLD,LLD-LLS,GR-GKUT
0,2705.2524,19.0270,18.728100,0.051300,5.2064,2.6527,0.019800,1,0.699109,0.749381,0.050272,0.298900
1,2705.4048,15.8497,16.507000,0.046800,5.2632,2.6564,0.022900,1,0.708140,0.769960,0.061820,-0.657300
2,2705.5572,13.3455,14.392100,0.041100,5.2742,2.6546,0.021800,1,0.751779,0.823416,0.071636,-1.046600
3,2705.7096,12.2315,13.696900,0.036900,5.2543,2.6488,0.020200,1,0.787269,0.865447,0.078178,-1.465400
4,2705.8620,13.0494,13.258900,0.032200,5.1625,2.6475,0.014500,1,0.833886,0.921738,0.087852,-0.209500
...,...,...,...,...,...,...,...,...,...,...,...,...
17456,2979.2676,24.4099,33.716596,0.368062,4.5393,2.2819,0.037705,3,-0.272377,-0.134245,0.138133,-9.306696
17457,2979.4200,24.4099,33.998501,0.368062,4.4872,2.2728,0.038547,3,-0.269783,-0.132356,0.137427,-9.588601
17458,2979.5724,24.4099,34.077848,0.368062,4.4662,2.2730,0.038599,3,-0.263922,-0.128077,0.135845,-9.667948
17459,2979.7248,24.4099,33.842925,0.368062,4.3735,2.2731,0.037318,3,-0.261378,-0.126215,0.135163,-9.433025


<h5 style="color:rgba(0, 0, 178, 0.6);">Split data to; train and test ; X and y </h5>

In [5]:
train = df[~df['well'].isin(df.groupby("well").get_group(6)['well'])]
train = train[~train['well'].isin(train.groupby("well").get_group(4)['well'])]
test = df.groupby("well").get_group(6)
val = df.groupby("well").get_group(4)

X_train = train.drop(["NPHI", "RHOB","DRHO", "well"], axis=1)
y_train = train["NPHI"]

X_test = test.drop(["NPHI", "RHOB", "DRHO", "well"], axis=1)
y_test = test["NPHI"]

X_val = val.drop(["NPHI", "RHOB", "DRHO", "well"], axis=1)
y_val = val["NPHI"]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

<h5 style="color:rgba(0, 0, 178, 0.6);">Create a function that build a dynamic model that can change the shape of the NN model with hyper parameters(hp) parameters</h5>

In [17]:
# hp ======> hyper parameters = [number_nuron_layer1, number_nuron_layer2, number_nuron_layer3,
#                          number_nuron_layer4, Activation_fuctin_layer1, Activaiton_function_layer2,
#                     Activation_fuctin_layer3, Activation_fuctin_layer4, If for layer1 have Drop_out=1 else 0,
#         If for layer2 have Drop_out=1 else 0, If for layer3 have Drop_out=1 else 0, If for layer4 have Drop_out=1 else 0,
#     If for layer1 have batch_norm=1 else 0, If for layer2 have batch_norm=1 else 0, If for layer1 have batch_norm=3 else 0,
#                       If for layer1 have batch_norm=1 else 0, learning_rate, batch_size]


# in this function we create a function that can build model that can chane evry time

def dynamic_model(hp, train=train):


    activation_function = ["relu","sigmoid","softmax","softsign",
                           "tanh","selu","elu" ,"exponential",
                           "leaky_relu","relu6","silu","gelu",
                           "hard_sigmoid","linear","mish","log_softmax"]
    

    input_layer = keras.layers.Input(shape=(8,))
    layer1 = keras.layers.Dense(hp[0], activation_function[int(hp[4])], name="layer1")(input_layer)
    
#     in this sectino we have 4 layer  so that we have 4 section for dscetion that we have in each layer DropOut or BatchNorm
#   this is layer1 we have 4 condition:
    if ((hp[8] == 1) & (hp[12] == 1)):
        drop1 = keras.layers.Dropout(0.3)(layer1)
        norm1 = keras.layers.BatchNormalization()(drop1)
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(norm1)
    elif ((hp[8] == 1) & (hp[12] == 0)):
        drop1 = keras.layers.Dropout(0.3)(layer1)
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(drop1)
    elif ((hp[8] == 0) & (hp[12] == 1)):
        norm1 = keras.layers.BatchNormalization()(layer1)
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(norm1)
    else:
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(layer1)
        
#    layer2:     
    if ((hp[9] == 1) & (hp[13] == 1)):
        drop2 = keras.layers.Dropout(0.3)(layer2)
        norm2 = keras.layers.BatchNormalization()(drop2)
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(norm2)
    elif ((hp[9] == 1) & (hp[13] == 0)):
        drop2 = keras.layers.Dropout(0.3)(layer2)
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(drop2)
    elif ((hp[9] == 0) & (hp[13] == 1)):
        norm2 = keras.layers.BatchNormalization()(layer2)
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(norm2)
    else:
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(layer2)
        
#     layer3 :         
    if ((hp[10] == 1) & (hp[14] == 1)):
        drop3 = keras.layers.Dropout(0.3)(layer3)
        norm3 = keras.layers.BatchNormalization()(drop3)
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(norm3)
    elif ((hp[10] == 1) & (hp[14] == 0)):
        drop3 = keras.layers.Dropout(0.3)(layer3)
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(drop3)
    elif ((hp[10] == 0) & (hp[14] == 1)):
        norm3 = keras.layers.BatchNormalization()(layer3)
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(norm3)
    else:
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(layer3)
        
#    layer4:        
    if ((hp[11] == 1) & (hp[15] == 1)):
        drop4 = keras.layers.Dropout(0.3)(layer4)
        norm4 = keras.layers.BatchNormalization()(drop4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(norm4)
    elif ((hp[11] == 1) & (hp[15] == 0)):
        drop4 = keras.layers.Dropout(0.3)(layer4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(drop4)
    elif ((hp[11] == 0) & (hp[15] == 1)):
        norm4 = keras.layers.BatchNormalization()(layer4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(norm4)
    else:
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(layer4)

#     at last we create final model        
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    
#     in this section we complie the model    
    optimizer =keras.optimizers.Adam(learning_rate=hp[16])
    model.compile(loss="mae", optimizer=optimizer, metrics=["mse"])

    # wright the callbacks that we need
#     checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints.h5", save_weights_only=True,best_only=True)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)
    
    # fit the model
    history = model.fit(X_train,
                         y_train, 
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping_cb],
                        batch_size=int(hp[17]),
                        epochs=1000,
                        verbose=0)
    
#     in this section we complie the model
    val_pred = model.predict(X_val)
#   because we r2_score dont work with nan values we chane nan value with -999
    val_pred[np.isnan(val_pred)] = -999
        
    solution_fitness = r2_score(y_val, val_pred)
    print(solution_fitness)
 
    return solution_fitness


<h5 style="color:rgba(0, 0, 178, 0.6);">Create a function that build randomly particle for obtimizer</h5>

In [18]:
def hp_creator(num_solutions=100):
    hps = pd.DataFrame(columns=[
                                "nurn1","nurn2","nurn3","nurn4",
                                "Active1","Active2","Active3","Active4",
                                "Drop1", "Drop2", "Drop3", "Drop4",
                                "Norm1", "Norm2", "Norm3", "Norm4",
                                "Learn_rate", "batch_size"
                               ])
    
    hps["nurn1"], hps["nurn2"] = [random.randint(10, 200) for _ in range(num_solutions)], [random.randint(10, 200) for _ in range(num_solutions)]
    hps["nurn3"], hps["nurn4"] = [random.randint(10, 200) for _ in range(num_solutions)], [random.randint(10, 200) for _ in range(num_solutions)]
    
    hps["Active1"], hps["Active2"] = [random.randint(0, 15) for _ in range(num_solutions)], [random.randint(0, 15) for _ in range(num_solutions)]
    hps["Active3"], hps["Active4"] = [random.randint(0, 15) for _ in range(num_solutions)], [random.randint(0, 15) for _ in range(num_solutions)]
                                                                                 
    hps["Drop1"], hps["Drop2"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    hps["Drop3"], hps["Drop4"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    
    hps["Norm1"], hps["Norm2"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    hps["Norm3"], hps["Norm4"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    
    hps["Learn_rate"] = [random.choice([0.001, 0.01, 0.0001, 0.00001, 0.1, 0.000001, 0.0000001]) for _ in range(num_solutions)]
    hps["batch_size"] = [random.randint(30, 300) for _ in range(num_solutions)]
    

    return hps

<h5 style="color:rgba(0, 0, 178, 0.6);">function that create PSO model and run it</h5>

In [19]:
# num_particles ==> number of rnadom input that generate
# max_iter ==> number of iteration that function use to solve the problem
# w and c1 and c2 ==> constant parameters this are hayper parameter


def pso(num_particles=100, max_iter=20, w=0.5, c1=1, c2=2):
    # Initialize particles and velocities
    particles = hp_creator(num_solutions=num_particles).values
    velocities = np.zeros((num_particles, len(particles[0,:])))

    # Initialize the best positions and fitness values
    best_positions = np.copy(particles)
    best_fitness = np.array([dynamic_model(p) for p in particles])
    swarm_best_position = best_positions[np.argmax(best_fitness)]
    swarm_best_fitness = np.max(best_fitness)
    
    # Iterate through the specified number of iterations, updating the velocity and position of each particle at each iteration
    for k in range(max_iter):
        # Update velocities
        r1 = np.random.uniform(0, 1, (num_particles, 18))
        r2 = np.random.uniform(0, 1, (num_particles, 18))
        velocities = w * velocities + c1 * r1 * (best_positions - particles) + c2 * r2 * (swarm_best_position - particles)

        # Update positions
        particles += velocities
# --------------------------------------------------------------------------------------------------------------
#       in this section we need to the updated positions in bonderys that we need so we set ruls to it
#       ferst for for check each particle and secend for to check each parameters
        for i, hp in enumerate(particles):
            for j, param in enumerate(hp):
                #------------------------------------------------------------------------------------------------
                #bounders for number of nurouns in each leayer
                if (j>=0)&(j<4):
                    while (10>particles[i,j])or(200<particles[i,j]):
                        if particles[i,j]<10:
                            particles[i,j] = 10 + (10-int(particles[i,j]))
                        elif particles[i,j]>200:
                            particles[i,j] = 200-(int(particles[i,j])-200)
                    particles[i,j] = int(particles[i,j])
               #--------------------------------------------------------------------------------------------------
                #bounders for which activation we should use    
                if (j>=4)&(j<8):
                    while (0>particles[i,j])or(15<particles[i,j]):
                        if particles[i,j]<0:
                            particles[i,j] = 0 - int(round(particles[i,j]))
                        elif particles[i,j]>15:
                            particles[i,j] = 15-(int(round(particles[i,j])-15))
                    particles[i,j] = int(round(particles[i,j]))
              #----------------------------------------------------------------------------------------------------
                #bounders for if we have Dropuout or Batchnormalize or not
                if (j>=8)&(j<16):
                    if particles[i,j]<=0.5:
                        particles[i,j] = 0
                    elif particles[i,j]>0.5:
                        particles[i,j] = 1
             #------------------------------------------------------------------------------------------------------
                #bounders for learning rate
                if j==16:
                    while (10e-7>particles[i,j])or(0.1<particles[i,j]):
                        if particles[i,j]<10e-7:
                            particles[i,j] = 10e-7 + (10e-7-particles[i,j])
                        elif particles[i,j]>0.1:
                            particles[i,j] = 0.1-(particles[i,j]-0.1)
            #------------------------------------------------------------------------------------------------------
                #bounders for batch size
                if j==17:
                    while (30>particles[i,j])or(300<particles[i,j]):
                        if particles[i,j]<30:
                            particles[i,j] = 30 + (30-int(particles[i,j]))
                        elif particles[i,j]>300:
                            particles[i,j] = 300-(int(particles[i,j])-300)
                    particles[i,j] = int(particles[i,j])
#------------------------------------------------------------------------------------------------------------------
                    
        # Evaluate fitness of each particle
        print("=========================================================================")
        print("iter:",k)
        fitness_values = np.array([dynamic_model(p) for p in particles])
        print("==========================================================================")
        # Update best positions and fitness values
        improved_indices = np.where(fitness_values > best_fitness)
        best_positions[improved_indices] = particles[improved_indices]
        best_fitness[improved_indices] = fitness_values[improved_indices]
        if np.max(fitness_values) > swarm_best_fitness:
            swarm_best_position = particles[np.argmax(fitness_values)]
            swarm_best_fitness = np.max(fitness_values)

    # Return the best solution found by the PSO algorithm
    return swarm_best_position, swarm_best_fitness


# Run the PSO algorithm on the Rastrigin function
solution, fitness = pso(max_iter=5)

# Print the solution and fitness value
print('Solution:', solution)
print('Fitness:', fitness)
    
    



38/38 [==============================] - 0s 2ms/step
-0.021080947220908586
38/38 [==============================] - 0s 1ms/step
-0.05921736510153086
38/38 [==============================] - 0s 1ms/step
-101.75155204425779
38/38 [==============================] - 0s 1ms/step
0.2744386459511856
38/38 [==============================] - 0s 1ms/step
-291936953.7702268
38/38 [==============================] - 0s 2ms/step
0.6383478434690266
38/38 [==============================] - 0s 1ms/step
0.685370463446493
38/38 [==============================] - 0s 1ms/step
-11.273767304135463
38/38 [==============================] - 0s 2ms/step
-1.2181212491155926
38/38 [==============================] - 0s 1ms/step
0.6228282400784859
38/38 [==============================] - 0s 1ms/step
-1.9417620897919865
38/38 [==============================] - 0s 1ms/step
-0.10486131703615897
38/38 [==============================] - 0s 1ms/step
-291936953.7702268
38/38 [==============================] - 0s 2ms/step

38/38 [==============================] - 0s 2ms/step
0.6742669683452283
38/38 [==============================] - 0s 2ms/step
0.6812986661726304
38/38 [==============================] - 0s 2ms/step
0.6916757569502846
38/38 [==============================] - 0s 2ms/step
0.648511645157549
38/38 [==============================] - 0s 2ms/step
0.6801174520395951
38/38 [==============================] - 0s 2ms/step
0.6437531845129767
38/38 [==============================] - 0s 2ms/step
0.6917078314060814
38/38 [==============================] - 0s 2ms/step
0.6643442429194452
38/38 [==============================] - 0s 2ms/step
0.6177019932149589
38/38 [==============================] - 0s 2ms/step
0.7066209721306735
38/38 [==============================] - 0s 2ms/step
0.5396657959170885
38/38 [==============================] - 0s 2ms/step
0.6493904136034265
38/38 [==============================] - 0s 2ms/step
0.6700486868977882
38/38 [==============================] - 0s 2ms/step
0.5295662698

0.6720939218863755
38/38 [==============================] - 0s 2ms/step
0.6848655102318602
38/38 [==============================] - 0s 2ms/step
0.6806690051769975
38/38 [==============================] - 0s 2ms/step
0.6511815229224673
38/38 [==============================] - 0s 2ms/step
0.614833533205324
38/38 [==============================] - 0s 2ms/step
0.659793808437193
38/38 [==============================] - 0s 2ms/step
0.644634988244339
38/38 [==============================] - 0s 1ms/step
0.6707760896923616
38/38 [==============================] - 0s 1ms/step
0.6032018376839472
38/38 [==============================] - 0s 4ms/step
0.7049337918088272
38/38 [==============================] - 0s 2ms/step
0.693373779347058
38/38 [==============================] - 0s 2ms/step
0.6622980762171147
38/38 [==============================] - 0s 2ms/step
0.703519302768904
38/38 [==============================] - 0s 2ms/step
0.6742641377611585
38/38 [==============================] - 0s 4ms/st

38/38 [==============================] - 0s 2ms/step
0.49882840469308554
38/38 [==============================] - 0s 2ms/step
0.6829307125544308
38/38 [==============================] - 0s 4ms/step
0.6558914886654139
38/38 [==============================] - 0s 2ms/step
0.700358735439381
38/38 [==============================] - 0s 2ms/step
0.6725398923878853
38/38 [==============================] - 0s 1ms/step
0.526594481432302
38/38 [==============================] - 0s 1ms/step
0.6730435599448517
38/38 [==============================] - 0s 2ms/step
0.5899738053382364
38/38 [==============================] - 0s 2ms/step
0.603880745101667
38/38 [==============================] - 0s 2ms/step
0.6416708559452007
38/38 [==============================] - 0s 2ms/step
0.6777351310197397
38/38 [==============================] - 0s 2ms/step
0.6927095030000641
38/38 [==============================] - 0s 2ms/step
0.6762702758169239
38/38 [==============================] - 0s 2ms/step
0.66613660896